## Environment Variables


In [ ]:
langchain                     0.3.18
langchain-community           0.3.17
langchain-core                0.3.34
langchain-nvidia-ai-endpoints 0.3.9
langchain-text-splitters      0.3.6
langgraph                     0.1.19
langsmith                     0.1.147

In [ ]:
aiohappyeyeballs              2.4.4
aiohttp                       3.11.11
aiosignal                     1.3.2
annotated-types               0.7.0
anyio                         4.8.0
asttokens                     3.0.0
attrs                         24.3.0
certifi                       2024.12.14
charset-normalizer            3.4.1
comm                          0.2.2
dataclasses-json              0.6.7
debugpy                       1.8.12
decorator                     5.1.1
executing                     2.2.0
frozenlist                    1.5.0
greenlet                      3.1.1
h11                           0.14.0
httpcore                      1.0.7
httpx                         0.28.1
httpx-sse                     0.4.0
idna                          3.10
ipykernel                     6.29.5
ipython                       8.31.0
jedi                          0.19.2
jsonpatch                     1.33
jsonpointer                   3.0.0
jupyter_client                8.6.3
jupyter_core                  5.7.2
langchain                     0.3.18
langchain-community           0.3.17
langchain-core                0.3.34
langchain-nvidia-ai-endpoints 0.3.9
langchain-text-splitters      0.3.6
langgraph                     0.1.19
langsmith                     0.1.147
marshmallow                   3.26.1
matplotlib-inline             0.1.7
multidict                     6.1.0
mypy-extensions               1.0.0
nest-asyncio                  1.6.0
numpy                         1.26.4
orjson                        3.10.15
packaging                     24.2
parso                         0.8.4
pexpect                       4.9.0
pillow                        10.4.0
pip                           24.3.1
platformdirs                  4.3.6
prompt_toolkit                3.0.50
propcache                     0.2.1
psutil                        6.1.1
ptyprocess                    0.7.0
pure_eval                     0.2.3
pydantic                      2.10.5
pydantic_core                 2.27.2
pydantic-settings             2.7.1
Pygments                      2.19.1
python-dateutil               2.9.0.post0
python-dotenv                 1.0.1
PyYAML                        6.0.2
pyzmq                         26.2.0
requests                      2.32.3
requests-toolbelt             1.0.0
setuptools                    65.5.0
six                           1.17.0
sniffio                       1.3.1
SQLAlchemy                    2.0.37
stack-data                    0.6.3
tenacity                      8.5.0
tornado                       6.4.2
traitlets                     5.14.3
typing_extensions             4.12.2
typing-inspect                0.9.0
urllib3                       2.3.0
wcwidth                       0.2.13
yarl                          1.18.3
zstandard                     0.23.0

In [4]:
import os

os.environ["NVIDIA_API_KEY"] = "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"

In [5]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="Curiosity"

In [6]:
os.environ["TAVILY_API_KEY"]="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"

Ref. to: Langgraph introduction https://langchain-ai.github.io/langgraph/tutorials/introduction/#requirements

## Persistent Memory Using Langchain's LangGraph

In [8]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Generate response
llm = ChatNVIDIA(
    model="nvdev/meta/llama-3.1-70b-instruct",
    nvidia_api_key=os.environ["NVIDIA_API_KEY"],
)

In [9]:
llm

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='nvdev/meta/llama-3.1-70b-instruct')

**TypedDict**: This is a way to define a dictionary with a fixed schema. Here, we’re saying that our state will always have a key called "messages".

**Annotated with add_messages**: Normally, when you update a key in a dictionary, you might replace its value. But thanks to the add_messages annotation, when new messages are returned, they are appended to the existing list. This is how our conversation history grows over time.

As a result, you get a Dictionary with the key {'messages': [list]} which allows appending of new messages instead of replacing the original value.

In [10]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import add_messages

class State(TypedDict):
    # The "messages" key is a list.
    # The `add_messages` annotation tells our system to **append** new messages
    # to this list instead of replacing the entire list.
    messages: Annotated[list, add_messages]

/root/curiosity/venv_nvdev/lib/python3.11/site-packages/langgraph/graph/graph.py:30: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langgraph.pregel import Channel, Pregel


**StateGraph(State)**: This initializes a state graph that knows the structure of your state (in this case the Dictionary). The graph manages how your state moves from one node (or step) to another.

**START and END**: These are special markers used to denote the entry and exit points of your workflow.

In [11]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(State)

**chatbot function**: This is a node in the graph.
- **Input**: It receives the current state, which includes the full conversation history in state["messages"].
- **Processing**: It calls llm.invoke(state["messages"]), which means it uses the language model (LLM) to generate a new message based on the conversation history.
- **Output**: It returns a dictionary with a new message inside a list. Because of the add_messages annotation on "messages", this new message gets appended to the existing conversation history rather than replacing it.

In [12]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

**add_node**: This registers your chatbot function as a node in the graph.

**add_edge (START to chatbot)**: This means the graph will start by running the chatbot node.

**add_edge (chatbot to END)**: Once the chatbot node finishes its task, the workflow is complete.

In [13]:
# Adding the chatbot node to our graph with a unique name.
graph_builder.add_node("chatbot", chatbot)

# Defining the flow: Start → Chatbot
graph_builder.add_edge(START, "chatbot")

# After the chatbot node, the workflow goes to the End.
graph_builder.add_edge("chatbot", END)


**MemorySaver**: This is a checkpoint mechanism. It saves the state after each node execution.

**compile**: This finalizes your graph, integrating the MemorySaver so that every state update is recorded.

In [14]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [17]:
thread_id = "testing_name"
config = {"configurable": {"thread_id": thread_id}}

user_input = "Hi! My name is Will"

# Initial state (with user's message) is fed into the workflow
# Chatbot node takes the current conversation and generates a response using llm.invoke, and then appends that response to "messages" list
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! My name is Will
================================== Ai Message ==================================

We're on a loop, Will! I think I've already welcomed you twice. Just to keep things fresh, would you like to tell me something about yourself, or perhaps play a game, or talk about a favorite hobby?


In [85]:
# Defining custom JSON serialisable to print the indents for the JSON response
def custom_serializer(obj):
    # If the object has a method to convert to a dict, use it.
    if hasattr(obj, "dict"):
        return obj.dict()
    # If it's a HumanMessage-like object, try to convert it manually.
    # Adjust the attribute names as needed for your specific object.
    if hasattr(obj, "role") and hasattr(obj, "content"):
        return {"role": obj.role, "content": obj.content}
    # Fallback: convert to string.
    return str(obj)

import json

In [46]:
user_input = "Remember my name?"

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

Your name is Will. I'll remember it for our conversation.

====================== What is happening behind the scenes:======================
{'v': 1, 'ts': '2025-01-23T08:15:41.775353+00:00', 'id': '1efd9623-c8e4-6518-8004-ca48147710d2', 'channel_values': {'messages': [HumanMessage(content='Hi! My name is Will', additional_kwargs={}, response_metadata={}, id='9e8fda90-591b-4c7e-8617-0ea29dcf6591'), AIMessage(content="Hi Will! It's nice to meet you! Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hi Will! It's nice to meet you! Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 16, 'total_tokens': 41, 'completion_tokens': 25}, 'finish_reason': 'stop', 'model_name': '

In [18]:
print("\n====================== What is happening behind the scenes:======================")
print(json.dumps(memory.get(config), indent=2, default=custom_serializer))


====================== What is happening behind the scenes:======================
{
  "v": 1,
  "ts": "2025-02-12T07:32:04.277138+00:00",
  "id": "1efe9137-4aab-6d3f-8007-29f3812d02f0",
  "channel_values": {
    "messages": [
      {
        "content": "Hi! My name is Will",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": "3b90ffa5-8501-4907-9b9a-d766a4a7dea0",
        "example": false
      },
      {
        "content": "Nice to meet you, Will! Is there something I can help you with or would you like to chat for a bit?",
        "additional_kwargs": {},
        "response_metadata": {
          "role": "assistant",
          "content": "Nice to meet you, Will! Is there something I can help you with or would you like to chat for a bit?",
          "token_usage": {
            "prompt_tokens": 16,
            "total_tokens": 41,
            "completion_tokens": 25
          },
          "finish_reason": "stop

/tmp/ipykernel_24310/1619559381.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return obj.dict()


In [24]:
def chatbot_with_logging(state: State):
    print("\n")
    print("Current conversation history:", state["messages"])
    print("\n")
    new_message = llm.invoke(state["messages"])
    print("New message from LLM:")
    print(json.dumps(new_message, indent=2, default=custom_serializer))
    print("\n")
    return {"messages": [new_message]}

graph_builder_with_logging = StateGraph(State)

# Adding the chatbot node to our graph with a unique name.
graph_builder_with_logging.add_node("chatbot", chatbot_with_logging)

# Defining the flow: Start → Chatbot
graph_builder_with_logging.add_edge(START, "chatbot")

# After the chatbot node, the workflow goes to the End.
graph_builder_with_logging.add_edge("chatbot", END)

memory_with_logging = MemorySaver()
graph_with_logging = graph_builder_with_logging.compile(checkpointer=memory_with_logging)

thread_id = "test3_with_logging"
config = {"configurable": {"thread_id": thread_id}}

user_input = "Hi! My name is Will"

# Initial state (with user's message) is fed into the workflow
# Chatbot node takes the current conversation and generates a response using llm.invoke, and then appends that response to "messages" list
events = graph_with_logging.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! My name is Will
Current conversation history: [HumanMessage(content='Hi! My name is Will', additional_kwargs={}, response_metadata={}, id='43374524-3192-4774-a211-a3606dffa725')]


New message from LLM:
{
  "content": "Nice to meet you, Will! Is there something I can help you with or would you like to chat for a bit?",
  "additional_kwargs": {},
  "response_metadata": {
    "role": "assistant",
    "content": "Nice to meet you, Will! Is there something I can help you with or would you like to chat for a bit?",
    "token_usage": {
      "prompt_tokens": 16,
      "total_tokens": 41,
      "completion_tokens": 25
    },
    "finish_reason": "stop",
    "model_name": "nvdev/meta/llama-3.1-70b-instruct"
  },
  "type": "ai",
  "name": null,
  "id": "run-ba580023-9709-4290-aded-f17209b3657e-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 16,

/tmp/ipykernel_24310/1619559381.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return obj.dict()


In [22]:
print("\n====================== What is happening behind the scenes:======================")
print(json.dumps(memory_with_logging.get(config), indent=2, default=custom_serializer))


====================== What is happening behind the scenes:======================
{
  "v": 1,
  "ts": "2025-02-12T07:37:18.334212+00:00",
  "id": "1efe9142-fdc0-6b23-8001-64978a3381b3",
  "channel_values": {
    "messages": [
      {
        "content": "Hi! My name is Will",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": "b655b6d8-de38-4923-b1db-b0dc2f79837b",
        "example": false
      },
      {
        "content": "Nice to meet you, Will! Is there something I can help you with or would you like to chat for a bit?",
        "additional_kwargs": {},
        "response_metadata": {
          "role": "assistant",
          "content": "Nice to meet you, Will! Is there something I can help you with or would you like to chat for a bit?",
          "token_usage": {
            "prompt_tokens": 16,
            "total_tokens": 41,
            "completion_tokens": 25
          },
          "finish_reason": "stop

/tmp/ipykernel_24310/1619559381.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return obj.dict()


### Using .invoke instead of .stream

In [54]:
thread_id = "testing_invoke"
config = {"configurable": {"thread_id": thread_id}}

result = graph.invoke(
    {"messages": [{"role": "user", "content": "What is the capital of France?"}]},
    config=config,
)
print(result["messages"][-1].content)

Consistency is key! The capital of France is Paris. (I'm starting to sense a pattern here...)


## React Agent with Tavily tool

In [59]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

search = TavilySearchResults(max_results=5, include_images=True)
tools = [search]
agent = create_react_agent(llm, tools)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "What are some tools for education?"}]})

print("\n================================== Ai Message ==================================")
print(response["messages"][-1].content)

print(json.dumps(response, indent=2, default=custom_serializer))

/root/curiosity/venv_nvdev/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:591: UserWarning: Model 'nvdev/meta/llama-3.1-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(



================================== Ai Message ==================================
Some tools for education include:

* Learning Management Systems (LMS)
* eLearning Authoring Tools
* eLearning Content Providers
* eLearning Course Marketplaces
* Online Language Providers
* eLearning Voice Actors
* eLearning Translation Providers
* Web Conferencing Software
* Project Management Software
* TED-Ed
* Classpoint
* Prodigy
* Google Classroom
* Zoom
* Duolingo
* Bloomz
* Generative AI for Educators
* Cloud solutions

These tools can be used to create online collaborative groups, administer and provide educational materials, measure student performance, communicate with parents, and facilitate communication between teachers and learners. They can also be used to create educational lessons, simplify and enhance the learning experience, and provide virtual learning platforms and tools for teachers and students.
{
  "messages": [
    {
      "content": "What are some tools for education?",
      "

/tmp/ipykernel_24310/1619559381.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return obj.dict()


Previously, this was the State (structured Dict created)

In [ ]:
class State(TypedDict):
    # The "messages" key is a list.
    # The `add_messages` annotation tells our system to **append** new messages
    # to this list instead of replacing the entire list.
    messages: Annotated[list, add_messages]

In order to use create_react_agent, there is another parameter you need:

**is_last_step**

- With is_last_step:
The agent clearly knows when to stop and output the final result.

- Without is_last_step:
The agent might continue to look for additional steps or fail to signal that the conversation or reasoning process has concluded.

In [48]:
from langchain.schema import BaseMessage

class TavilyState(TypedDict):
        messages: Annotated[list, add_messages]
        is_last_step: str

Creating a new function to print the stream, taking into account the tool calling different printing (unable to do automatic pretty_print)

In [76]:
def print_stream(graph, inputs, config):
    for s in graph.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple): # To enable tool calling printing
            print(message)
        else:
            message.pretty_print()

Creating the react agent, which implements the graphing logic previously explored within the function.

In [80]:
search = TavilySearchResults(max_results=5, include_images=True)
tools = [search]
memory_tavily = MemorySaver()

graph_with_tavily = create_react_agent(llm, tools, state_schema=TavilyState, checkpointer=memory_tavily)

/root/curiosity/venv_nvdev/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:591: UserWarning: Model 'nvdev/meta/llama-3.1-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(


In [81]:
thread_id = "test10_with_tavily"
config = {"configurable": {"thread_id": thread_id}}

inputs = {"messages": [("user", "Hi, I'm Will! Nice to meet you. I'm a teacher.")]}
print_stream(graph_with_tavily, inputs, config)

================================ Human Message =================================

Hi, I'm Will! Nice to meet you. I'm a teacher.
================================== Ai Message ==================================

Nice to meet you too, Will! It's great to hear that you're a teacher. What subject do you teach, if I might ask?


No tool use is called. Let's look at what's happening behind the scenes

In [82]:
print("\n====================== What is happening behind the scenes:======================")
print(json.dumps(memory_tavily.get(config), indent=2, default=custom_serializer))


====================== What is happening behind the scenes:======================
{
  "v": 1,
  "ts": "2025-02-12T09:26:52.116666+00:00",
  "id": "1efe9237-e23a-64d8-8001-ece5de051bce",
  "channel_values": {
    "messages": [
      {
        "content": "Hi, I'm Will! Nice to meet you. I'm a teacher.",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": "407fab28-eaa9-4a10-9451-a9786f5688c1",
        "example": false
      },
      {
        "content": "Nice to meet you too, Will! It's great to hear that you're a teacher. What subject do you teach, if I might ask?",
        "additional_kwargs": {},
        "response_metadata": {
          "role": "assistant",
          "content": "Nice to meet you too, Will! It's great to hear that you're a teacher. What subject do you teach, if I might ask?",
          "token_usage": {
            "prompt_tokens": 323,
            "total_tokens": 353,
            "completion_to

/tmp/ipykernel_24310/1619559381.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return obj.dict()


In [83]:
inputs = {"messages": [("user", "Can you give me some articles on how AI can help me with my job?")]}
print_stream(graph_with_tavily, inputs, config)

================================ Human Message =================================

Can you give me some articles on how AI can help me with my job?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (chatcmpl-tool-632f68d68a494905aac8f624cddf85d2)
 Call ID: chatcmpl-tool-632f68d68a494905aac8f624cddf85d2
  Args:
    query: AI for teachers
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.edutopia.org/article/7-ai-tools-that-help-teachers-work-more-efficiently", "content": "Not only can AI tools enhance creativity and productivity, but also they can provide educators with valuable insights into student learning and assist with some of the time-consuming tasks that educators have. The right AI tools can help to automate or streamline these tasks, which allows teachers to have additional time with their students. Here are seven AI-powe

In [84]:
print("\n====================== What is happening behind the scenes:======================")
print(json.dumps(memory_tavily.get(config), indent=2, default=custom_serializer))


====================== What is happening behind the scenes:======================
{
  "v": 1,
  "ts": "2025-02-12T09:28:27.640753+00:00",
  "id": "1efe923b-7137-66a7-8006-7d442f834cb6",
  "channel_values": {
    "messages": [
      {
        "content": "Hi, I'm Will! Nice to meet you. I'm a teacher.",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": "407fab28-eaa9-4a10-9451-a9786f5688c1",
        "example": false
      },
      {
        "content": "Nice to meet you too, Will! It's great to hear that you're a teacher. What subject do you teach, if I might ask?",
        "additional_kwargs": {},
        "response_metadata": {
          "role": "assistant",
          "content": "Nice to meet you too, Will! It's great to hear that you're a teacher. What subject do you teach, if I might ask?",
          "token_usage": {
            "prompt_tokens": 323,
            "total_tokens": 353,
            "completion_to

/tmp/ipykernel_24310/1619559381.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return obj.dict()
